### CH06 Getting More out of our Objects With Descriptors

:파이썬의 고급기능 디스크립터 다른언어에는 생소한 개념

#### 디스크립터 개요

디스크립터 메커니즘(descriptor mechanism)
- 최소 두 개의 클래스가 필요
- 디스크립터 프로토콜 중 최소 한개 이상 클래스 속성으로 포함해야 함

(python 3.6기준 디스크립터 프로토콜)

- __get__
- __set__
- __delete__
- __set_name__

Tip : 디스크립터 객체는 항상 클래스 속성으로 선언해야 한다!

In [1]:
""" 디스크립터는 __get__ 매직 메서드의 결과를 반환함 """

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner): # owner의 인스턴스와 클래스를 받음
        if instance is None:
            return self
        logger.info(
            "Call: %s.__get__(%r, %r)", 
            self.__class__.__name__,
            instance,
            owner,
        )
        return instance


class ClientClass:
    descriptor = DescriptorClass()

In [2]:
client = ClientClass()
client.descriptor

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x103448dc0>, <class '__main__.ClientClass'>)


In [3]:
client.descriptor is client

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x103448dc0>, <class '__main__.ClientClass'>)


True

#### 디스크립터 프로토콜의 메서드 검색

 _ _ get _ _(self, isinstance, owner)

- owner가 있는 경우는 클래스에서 직접 호출하는 경우 isinstance가 None이기 떄문

In [4]:
# descripters_methods_1.py

""" 디스크립터를 인스턴스로 호출할 때와 클래스 속성으로 호출할 때의 차이점 """

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self, owner
        logger.info("Call: %s.__get__(%r, %r)",
                    self.__class__.__name__, instance, owner)
        return instance
    
class ClientClass:
    descriptor = DescriptorClass()

In [7]:
ClientClass.descriptor

(<__main__.DescriptorClass at 0x103463e80>, __main__.ClientClass)

In [8]:
ClientClass().descriptor # instance가 None이므로 self를 반환

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x103463850>, <class '__main__.ClientClass'>)


##### _ _ set _ _(self, instance, value)

- 디스크립터에 값을 할당할 때 호출되며, _ _set_ _( ) 메서드를 구현한 디스크립터에 대해서만 활성화
-  _ _set_ _ 메서드를구현했는지 반드시 확인하여 부작용이생기지않도록 주의해야한다.

In [9]:
from typing import Callable, Any # 형 힌트 지원
                                 # Callable[[Arg1Type, Arg2Type], ReturnType] 

class Validation: # 검증함수와 에러 메세지만을 가지고 있고 value는 __set__ 호출 시 전달받음
    """A configurable validation callable."""

    def __init__(
        self, validation_function: Callable[[Any], bool], error_msg: str 
    ) -> None:
        self.validation_function = validation_function
        self.error_msg = error_msg

    def __call__(self, value): # client.descriptor = 42 에서 42
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")


class Field:
    """A class attribute with validation functions configured over it."""
    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value): # client.descriptor = 42 처럼 대입 시 실행됨
        self.validate(value) # 여기서 raise ValueError 안된다면 다음 줄에서 대입됨
        instance.__dict__[self._name] = value


class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "is not a number"),
        Validation(lambda x: x >= 0, "is not >= 0"),
    )

In [10]:
client = ClientClass()
client.descriptor = 42
print(client.descriptor)

client.descriptor = -42

42


ValueError: -42 is not >= 0

In [11]:
client.descriptor = "string"

ValueError: 'string' is not a number

._s..et_0메서드가@property,setter가하던 일을대신

##### _ _ delete _ _(self, instance)

self 는 descriptor 속성을 나타내며, instance 는 client 를 나타낸다.

In [12]:
# descripters_methods_3.py

class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, user, value):
        if value is None:
            raise ValueError(f"{self._name} can't be set to None")
        user.__dict__[self._name] = value

    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(
                f"User {user!s} doesn't have {self.permission_required} "
                "permission"
            )


class User:
    """admin 권한이 있는 유저만 이메일 주소를 지울 수 있음"""

    email = ProtectedAttribute(requires_role="admin")

    def __init__(
        self, username: str, email: str, permission_list: list = None
    ) -> None:
        self.username = username
        self.email = email
        self.permissions = permission_list or []

    def __str__(self):
        return self.username